# **FIAP Pós-Tech - IA para Devs**

## TURMA 1IADT - GRUPO 14 - ALUNOS

* Marcelo Henriques da Fonseca - RM353865
* Marcos Lopes da Silva Junior - RM353763
* Ricardo Báfica Pontes - RM353866

## HACKATHON

### DETECÇÃO DE MATERIAIS CORTANTES

A FIAP VisionGuard, empresa de monitoramento de câmeras de segurança, está
analisando a viabilidade de uma nova funcionalidade para otimizar o seu software.

O objetivo da empresa é usar de novas tecnologias para identificar situações atípicas e
que possam colocar em risco a segurança de estabelecimentos e comércios que utilizam
suas câmeras.

Um dos principais desafios da empresa é utilizar Inteligência Artificial para identificar
objetos cortantes (facas, tesouras e similares) e emitir alertas para a central de
segurança.

A empresa tem o objetivo de validar a viabilidade dessa feature, e para isso será
necessário fazer um MVP para detecção supervisionada desses objetos.


### OBJETIVOS

*   Construir ou buscar um dataset contendo imagens de facas, tesouras e outros
objetos cortantes em diferentes condições de ângulo e iluminação.
*   Anotar o dataset para treinar o modelo supervisionado, incluindo imagens
negativas (sem objetos perigosos) para reduzir falsos positivos.
*   Treinar o modelo
*   Desenvolver um sistema de alertas (pode ser e-mail)


### ENTREGÁVEL

*   Documentação detalhando o fluxo utilizado para o desenvolvimento da solução
*   Vídeo de até 15 minutos explicando a solução proposta
*   Link do github do projeto


## TREINAMENTO

Neste notebook, detalharemos o processo de montagem e acesso do dataset e realizaremos o treinamento do modelo que será utilizado para detecção de objetos cortantes.

In [ ]:
# Instalar o Ultralytics e Roboflow
!pip install ultralytics roboflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.5/913.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.6 MB/s eta 0:00:00


### DATASET

De acordo com o objetivo da atividade, montamos um dataset com objetos cortantes para realização do treinamento do modelo.

Esse dataset contém uma variedade de imagens de objetos cortantes, além de imagens negativas (sem objetos perigosos) e imagens de garfos e colheres, a fim de reduzir falsos positivos.

Para a montagem do dataset, anotamos diversas imagens encontras online e também pegamos imagens já anotadas de outros datasets que encontramos.

Ao todo, são 4821 imagens que serão utilizadas no treinamento do modelo.

Por fim, decidimos usar o site Roboflow para fazer o controle dessas imagens.

O dataset montado para realização do treinamento pode ser encontrado no seguinte link:

https://app.roboflow.com/test-zeerx/knife_detection-roiuv

Datasets utilizados para a coletas de imagens anotadas:

https://storage.googleapis.com/openimages/web/index.html

https://datasetninja.com/od-weapon-detection-knife-detection

https://www.kaggle.com/datasets/kilianovski/spoonvsfork

https://universe.roboflow.com/dataset-pizme/axe-0eehv

https://universe.roboflow.com/projetos-ifsul/axes-detection

https://universe.roboflow.com/labelling-giajc/gun-knife-person

https://universe.roboflow.com/pa-uzup9/guntingdetect

Antes de começar o treinamento, nos conectaremos ao projeto hospedado no Roboflow a fim de baixar o dataset que será utilizado.

Para isso, precisamos fazer uso de uma API KEY e apontar corretamente o projeto, a versão do dataset que será baixada e o formato, que nesse caso será "yolov8".

In [ ]:
from roboflow import Roboflow
from google.colab import userdata

# Configurar chave da API Roboflow
api_key = userdata.get('ROBOFLOW_API_KEY') # Chave armazenada no "Secrets" do Google Colab

# Conectar ao projeto e baixar o dataset
rf = Roboflow(api_key=api_key)
project = rf.workspace("test-zeerx").project("knife_detection-roiuv")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to knife_detection-3 in yolov8:: 100%|██████████| 9648/9648 [00:01<00:00, 7121.47it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### TREINANDO O MODELO

Para este projeto decidimos fazer uso do [YoloV8](https://docs.ultralytics.com/models/yolov8/), um modelo da família YOLO para detecção de objetos em imagens e vídeos.

O YoloV8 possui uma alta precisão, velocidade e facilidade de uso. Ele é amplamente utilizado em atividades que exigem detecção em tempo real, oque o torna ideal para este projeto que demanda um processamento rápido e eficiente.

Dentre as versões disponíveis, optamos pelo "YOLOv8m", a versão de tamanho médio do modelo.

Ela oferece que um bom equilíbrio entre velocidade de treinamento e precisão, nos permitindo gerar modelos de maneira não tão demorada mas ao mesmo tempo não compromendo a qualidade.

Para o treinamento do nosso modelo, acessamos o arquivo data.yaml, que contém as configurações do dataset baixado localmente.

Neste arquivo podemos encontrar os caminhos para os diretórios contendo as imagens e anotações e as classes serão reconhecidas, que nesse caso são:

*   knife
*   axe
*   scissors
*   fork
*   spoon

O principal objetivo desse projeto é a detecção de objetos cortantes, que seriam as classes "knife", "axe" e "scissors".

Também definimos alguns hiperparâmetros:

**"Epochs":** define o número de épocas de treinamento. Uma época se refere a um ciclo que o modelo realizará processando o conjunto de treinamento.

Um número maior de épocas pode melhorar melhorar o aprendizado e a precisão do modelo, mas também requer um tempo maior de treino e pode levar a overfitting (quando o modelo se ajusta tão bem ao conjunto de treinamento que ele perde precisão ao detectar imagens nunca vistas antes).

**"imgsz":** tamanho das imagens utilizadas no treinamento.

Um tamanho maior pode melhorar a precisão, especialmente para detecção de objetos pequenos, mas também requer um maior uso de memória e tempo de treinamento.

**"Batch":** define o número de imagens processadas antes de atualizar os pesos do modelo.

Um número maior pode ajudar a estabilizar o treinamento, mas requer mais memória.

**"Device":** especifcia o dispositivo que será usado para o treinamento, CPU ou GPU.

In [ ]:
from ultralytics import YOLO

# Carregar o modelo YOLO
model = YOLO('yolov8m.pt')

# Iniciar o treinamento
model.train(
    data='/content/knife_detection-3/data.yaml',  # Caminho do arquivo data.yaml
    epochs=200,                         # Número de épocas
    imgsz=640,                          # Tamanho das imagens
    device=0                            # Use GPU (0 para a primeira GPU), 'cpu' para CPU
)

Ao final do treinamento, exportamos o modelo e o armazenamos em uma pasta no Google Drive, que pode ser entrada no link a seguir. Esse ambiente também armazena vídeos que extraímos da internet para testar o modelo, assim como os outputs dos testes.

https://drive.google.com/drive/folders/1t4HuKUUmmd-J29CkKJUHSI6jpNe_i49v?usp=sharing